In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 開始使用翻譯

<table align="left">
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fdoggy8088%2Fgenerative-ai%2Fmain%2Flanguage%2Ftranslation%2Fintro_translation.zh.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Vertex AI 圖像"><br> 在 Colab Enterprise 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/translation/intro_translation.zh.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI 圖像"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/translation/intro_translation.zh.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="Github 圖像"><br> 在 GitHub 上檢視
    </a>
  </td>
</table>


---

* 作者：Holt Skinner
* 建立時間：2024 年 1 月

---

## 概觀

本筆記本示範如何使用 [Google Cloud Translation API](https://cloud.google.com/translate) 翻譯 [130 多種語言](https://cloud.google.com/translate/docs/languages) 的文字。


### 目標

本教學課程使用以下 Google Cloud AI 服務與資源：

- [Cloud Translation API](https://cloud.google.com/translate/docs/overview)
- Cloud Storage


### 成本

本教學課程使用 Google Cloud 的計費元件：

* 翻譯服務
* Cloud Storage

了解 [翻譯服務價格](https://cloud.google.com/translate/pricing)，
以及 [Cloud Storage 價格](https://cloud.google.com/storage/pricing)，
並使用 [定價計算器](https://cloud.google.com/products/calculator/)，
根據預期的使用量進行成本估算。


## 開始使用


### 安裝 Vertex AI SDK、其他套件及其相依性

安裝執行此雲端筆記本所需的下列套件。


In [ ]:
# Install the packages
%pip install --user --upgrade -q google-cloud-translate

### 執行下列單元以重新啟動核心。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 這個 kernel 將會重新啟動。請等到重新啟動程序完成後再進行下一步。⚠️</b>
</div>


設定專案與地區。

* 請注意翻譯的**可用地區** ，請參閱文件[說明](https://cloud.google.com/translate/docs/advanced/endpoints)


In [5]:
PROJECT_ID = "YOUR_PROJECT_ID"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### 驗證筆記本環境

* 如果你使用 **Colab** 執行這個筆記本，請執行下列Cell並繼續。
* 如果你使用 **Vertex AI Workbench** ，請查看 [這裡](https://github.com/doggy8088/generative-ai/tree/main/setup-env) 的設定說明。


In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

    ! gcloud config set project {PROJECT_ID}
    ! gcloud auth application-default login -q

初始化 [Vertex AI Python SDK](https://cloud.google.com/vertex-ai/docs/python-sdk/use-vertex-ai-python-sdk)


In [11]:
import vertexai

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=VERTEXAI_LOCATION)

### 匯入函式庫


In [2]:
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.cloud import translate

### 建立客戶端


In [3]:
client = translate.TranslationServiceClient(
    # Optional: https://cloud.google.com/translate/docs/advanced/endpoints
    # client_options=ClientOptions(
    #     api_endpoint=f"translate-{TRANSLATE_LOCATION}.googleapis.com"
    # )
)

### 建立輔助函式


In [6]:
def translate_text(
    text: str,
    project_id: str = PROJECT_ID,
    location: str = LOCATION,
    glossary: Optional[str] = None,
) -> translate.TranslateTextResponse:
    """Translating Text."""
    # Translate text from English to Spanish
    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats
    response = client.translate_text(
        request=translate.TranslateTextRequest(
            parent=client.common_location_path(project_id, location),
            contents=[text],
            # Supported language codes: https://cloud.google.com/translate/docs/languages
            source_language_code="en",
            target_language_code="es",
            glossary_config=translate.TranslateTextGlossaryConfig(glossary=glossary)
            if glossary
            else None,
        )
    )

    return response


def create_glossary(
    input_uri: str,
    glossary_id: str,
    project_id: str = PROJECT_ID,
    location: str = LOCATION,
    timeout: int = 180,
) -> translate.Glossary:
    """
    Create a unidirectional glossary. Glossary can be words or
    short phrases (usually fewer than five words).
    https://cloud.google.com/translate/docs/advanced/glossary#format-glossary
    """
    glossary = translate.Glossary(
        name=client.glossary_path(project_id, location, glossary_id),
        # Supported language codes: https://cloud.google.com/translate/docs/languages
        language_pair=translate.Glossary.LanguageCodePair(
            source_language_code="en", target_language_code="es"
        ),
        input_config=translate.GlossaryInputConfig(
            gcs_source=translate.GcsSource(input_uri=input_uri)
        ),
    )

    # glossary is a custom dictionary Translation API uses
    # to translate the domain-specific terminology.
    operation = client.create_glossary(
        parent=client.common_location_path(project_id, location), glossary=glossary
    )

    result = operation.result(timeout)
    return result

現在讓我們嘗試將一個簡單的短語從英文翻譯成西班牙文。


In [11]:
response = translate_text("Hi there!")

# Display the translation for each input text provided
for translation in response.translations:
    print(f"Translated text: {translation.translated_text}")

Translated text: ¡Hola!


## 詞彙表


看起來不錯！但是，如果我們嘗試翻譯技術詞彙，例如 Google Cloud 產品 [運算引擎](https://cloud.google.com/compute?hl=en)，會發生什麼事？


In [21]:
response = translate_text("Compute Engine")

# Display the translation for each input text provided
for translation in response.translations:
    print(f"Translated text: {translation.translated_text}")

Translated text: Motor de Computación


### 建立詞彙表


注意，翻譯 API 逐字翻譯名稱。

假設我們希望這個名稱在所有語言中保持一致，我們可以建立 [詞彙表](https://cloud.google.com/translate/docs/advanced/glossary) 以一致翻譯特定網域的字詞和片語。

接下來，我們將為許多 Google Cloud 產品名稱建立詞彙表，以表示如何將它們翻譯成西班牙語。

我們已經建立一個輸入 TSV 檔案並上傳到公開存取的 Cloud Storage 儲存空間。


In [32]:
glossary = create_glossary(
    input_uri="gs://github-repo/translation/GoogleCloudGlossary.tsv",
    glossary_id="google_cloud_english_to_spanish",
)
print(glossary)

現在，讓我們試著用詞彙表再次翻譯文字。


In [10]:
response = translate_text("Compute Engine", glossary=glossary.name)

# Display the translation for each input text provided
for translation in response.translations:
    print(f"Default Translated text: {translation.translated_text}")

for translation in response.glossary_translations:
    print(f"Glossary Translated text: {translation.translated_text}")

Default Translated text: Motor de Computación
Glossary Translated text: Compute Engine
